In [1]:
#_*_coding: utf-8_*_
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import DOUBLE_PRECISION
import sqlalchemy
import psycopg2
import time
import os

In [2]:
conn_string = os.environ['PG_DB']
cur_path = os.getcwd()
ser = Service(cur_path+'/chromedriver')

In [3]:
df_k = pd.read_excel("keywords.xlsx", sheet_name=None)
list_word=[]
for project in df_k.keys():
 #   display(df_k[project]['Статистика по словам'][0])
    list_word.append(df_k[project]['Статистика по словам'][0])
#display(list_word)

In [4]:
options = webdriver.ChromeOptions()
# get source code
df_list=[]
for keyword in list_word:
    options.add_argument("--user-data-dir="+cur_path+"/profile")
    browser=webdriver.Chrome(service=ser, options=options)
    browser.get(r'https://wordstat.yandex.ru/#!/history?period=weekly&words='+keyword)
    time.sleep(15)
    html = browser.page_source
    time.sleep(1)
    browser.close()
    dfs = pd.read_html(html)
    df_1 = dfs[8]
    df_1 = df_1.drop(df_1.columns[[1,3]], axis = 1)
    df_1 = df_1.drop(0, axis = 0)
    df_2 = dfs[10]
    df_2 = df_2.drop(df_2.columns[[1,3]], axis = 1)
    df_2 = df_2.drop(0, axis = 0)
    df = pd.concat([df_1, df_2])
    df = df.reset_index()
    df["request"] = keyword
    df = df.drop(df.columns[[0]], axis = 1)
    df_list.append(df)
    #display(df)
    #df.to_csv(keyword.replace(" ", "")+'_week.csv' , sep=";",encoding='utf-8-sig')
df_all=pd.concat(df_list)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.177)
Stacktrace:
0   chromedriver                        0x00000001053d1138 chromedriver + 4923704
1   chromedriver                        0x00000001053499d3 chromedriver + 4368851
2   chromedriver                        0x0000000104f92787 chromedriver + 472967
3   chromedriver                        0x0000000104f68c15 chromedriver + 302101
4   chromedriver                        0x00000001050054cf chromedriver + 943311
5   chromedriver                        0x000000010501a2c6 chromedriver + 1028806
6   chromedriver                        0x0000000104fffd23 chromedriver + 920867
7   chromedriver                        0x0000000104fc6a4b chromedriver + 686667
8   chromedriver                        0x0000000104fc8044 chromedriver + 692292
9   chromedriver                        0x000000010539c8fe chromedriver + 4708606
10  chromedriver  

In [4]:
df_all['date_from']= df_all['Период'].str.split('-', n=1, expand=True)[0].str.strip()
df_all['date_to']= df_all['Период'].str.split('-', n=1, expand=True)[1].str.strip()
df_all.date_from = pd.to_datetime(df_all.date_from,  format = '%d.%m.%Y')
df_all.date_to = pd.to_datetime(df_all.date_to, format = '%d.%m.%Y')

In [45]:
# data from wordstat in df_all
#display(df_all)
df_all = df_all.reset_index()

In [5]:
db = create_engine(conn_string)
conn = db.connect()

In [6]:
print("Reading yt_reels_stat table with latest date...")
df_old =  pd.read_sql("SELECT * FROM wordstat_weekly", conn, parse_dates=['date_from', 'date_to'])
print("Done.")

Reading yt_reels_stat table with latest date...
Done.


In [58]:
df_res = pd.concat([df_old, df_all])

In [65]:
df_res = df_res.drop_duplicates(subset = ['Период', 'request'], keep = 'last') 

In [68]:
#display(df_res)

In [67]:
inject_dict = { 'Период': sqlalchemy.types.TEXT(),  
                'Абсолютное': sqlalchemy.types.BIGINT(), 
                'request': sqlalchemy.types.TEXT(),  
                'date_from': sqlalchemy.types.Date(),                   
                'date_to': sqlalchemy.types.Date(), 
                 }
start_time = time.time()
res = df_res.to_sql('wordstat_weekly', con=conn, if_exists='replace', index=False, dtype = inject_dict)
print("to_sql duration: {} seconds".format(time.time() - start_time))

to_sql duration: 5.560671091079712 seconds


In [70]:
display(df_all)

,Период,Абсолютное,request,date_from,date_to
0,14.02.2022 - 20.02.2022,27161.0,оранжевая корова,2022-02-14,2022-02-20
1,21.02.2022 - 27.02.2022,26896.0,оранжевая корова,2022-02-21,2022-02-27
2,28.02.2022 - 06.03.2022,24701.0,оранжевая корова,2022-02-28,2022-03-06
3,07.03.2022 - 13.03.2022,24578.0,оранжевая корова,2022-03-07,2022-03-13
4,14.03.2022 - 20.03.2022,25301.0,оранжевая корова,2022-03-14,2022-03-20
...,...,...,...,...,...
3999,09.01.2023 - 15.01.2023,1905.0,юху +и +его друзья,2023-01-09,2023-01-15
4000,16.01.2023 - 22.01.2023,1657.0,юху +и +его друзья,2023-01-16,2023-01-22
4001,23.01.2023 - 29.01.2023,1723.0,юху +и +его друзья,2023-01-23,2023-01-29
4002,30.01.2023 - 05.02.2023,1807.0,юху +и +его друзья,2023-01-30,2023-02-05
